# Рекомендация тарифов

**Описание проекта**

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**План и цели работы**

[1. Изучение предоставленных данных.](#section1)

[2. Разбивка данных на выборки.](#section2)

[3. Исследование моделей.](#section3)

- [Дерево предсказаний](#section3.1)
- [Случайный лес](#section3.2)
- [Логистическая регрессия](#section3.3)

[4. Проверка моделей на тестовой выборке.](#section4)

[5. Проверка модели на адекватность.](#section5)

[6. Общий вывод.](#section6)

<a id='section1'></a>
## Изучение предоставленных данных

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
# Импорт функции из бибилиотеки sklearn
from sklearn.model_selection import train_test_split
# Импорт метода дерева принятия решений
from sklearn.tree import DecisionTreeClassifier
# Импорт метода оценки доли правильных ответов
from sklearn.metrics import accuracy_score
# Импортируем алгоритм случайного леса
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import random

In [2]:
# сохраним данные в переменную
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# посмотрим на таблицу и информацию по ней
display(df.sample(5))
display(df.info())

,calls,minutes,messages,mb_used,is_ultra
2755,63.0,543.38,17.0,17229.84,0
719,1.0,8.33,5.0,2346.71,0
1972,5.0,13.39,0.0,2292.07,0
286,45.0,314.64,14.0,7332.79,0
2686,32.0,218.95,0.0,36180.51,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

**Вывод:**

- пропуски отсутствуют
- типы данных соответствуют
- в предобработке данные не нуждаются

<a id='section2'></a>
## Разбивка данных на выборки

Разобьём данные на три выборки: *обучающую*, *валидационную* и *тестовую* в отношении 3-1-1 с помощью метода train_test_split().

In [4]:
# Поделим датафрейм на обучающую выборку и выборку, которую позже разделим на валидационную и тестовую
df_train, df_divide = train_test_split(df, test_size = 0.40, random_state = 12345)

# Поделим df_divide валидационную и тестовую выборку
df_valid, df_test = train_test_split(df_divide, test_size = 0.50, random_state = 12345)

In [5]:
print(len(df_train))
print(len(df_valid))
print(len(df_test))

1928
643
643


**Вывод:** выборки были разделены.

<a id='section3'></a>
## Исследование моделей

Разделим наши датафреймы на целевой признак который нужно предсказать и признаки, которые помогут нам его предсказать:

In [6]:
# features и target - целевой признак
# обучающего датафрейма
features_train = df_train.drop(['is_ultra'], axis = 1)
target_train = df_train['is_ultra']

# валидационного датафрейма
features_valid = df_valid.drop(['is_ultra'], axis = 1)  
target_valid = df_valid['is_ultra']

# тестового датафрейма
features_test = df_valid.drop(['is_ultra'], axis = 1)
target_test = df_valid['is_ultra']

<a id='section3.1'></a>
**Модель дерево предсказаний**

In [7]:
for depth in range(1,30,2):
    model_tree = DecisionTreeClassifier(random_state = 12345, max_depth = depth) # имя
    
    model_tree.fit(features_train, target_train) # обучение
    
    prediction_tree = model_tree.predict(features_valid) # проверка
    
    print('max_depth:',depth,end='')
    print(' accuracy:',accuracy_score(prediction_tree,target_valid))

max_depth: 1 accuracy: 0.7542768273716952
max_depth: 3 accuracy: 0.7853810264385692
max_depth: 5 accuracy: 0.7791601866251944
max_depth: 7 accuracy: 0.7822706065318819
max_depth: 9 accuracy: 0.7822706065318819
max_depth: 11 accuracy: 0.7620528771384136
max_depth: 13 accuracy: 0.7558320373250389
max_depth: 15 accuracy: 0.7465007776049767
max_depth: 17 accuracy: 0.7356143079315708
max_depth: 19 accuracy: 0.7278382581648523
max_depth: 21 accuracy: 0.7278382581648523
max_depth: 23 accuracy: 0.7169517884914464
max_depth: 25 accuracy: 0.713841368584759
max_depth: 27 accuracy: 0.713841368584759
max_depth: 29 accuracy: 0.713841368584759


При max_depth = 3, accuracy = 0.7853810

<a id='section3.2'></a>
**Алгорит случайного леса**

In [8]:
for estim in range(1,30,2):    
    model_forest=RandomForestClassifier(max_depth = 9, n_estimators = estim, 
                                        random_state = 12345, min_samples_leaf = 5) # имя
    
    model_forest.fit(features_train, target_train) # обучение
    
    prediction_forest=model_forest.predict(features_valid) # проверка
    
    print('n_estimators:',estim,end="")
    print(' accuracy:',accuracy_score(prediction_forest,target_valid))

n_estimators: 1 accuracy: 0.7713841368584758
n_estimators: 3 accuracy: 0.7869362363919129
n_estimators: 5 accuracy: 0.7853810264385692
n_estimators: 7 accuracy: 0.7900466562986003
n_estimators: 9 accuracy: 0.7947122861586314
n_estimators: 11 accuracy: 0.7900466562986003
n_estimators: 13 accuracy: 0.7962674961119751
n_estimators: 15 accuracy: 0.80248833592535
n_estimators: 17 accuracy: 0.8009331259720062
n_estimators: 19 accuracy: 0.8009331259720062
n_estimators: 21 accuracy: 0.8009331259720062
n_estimators: 23 accuracy: 0.80248833592535
n_estimators: 25 accuracy: 0.7993779160186625
n_estimators: 27 accuracy: 0.7978227060653188
n_estimators: 29 accuracy: 0.7978227060653188


При max_depth = 9, max_samples_leaf = 5 и n_estimators= 15 (лучшее значение), accuracy = 0.8024883

<a id='section3.3'></a>
**Логостическая регрессия**

In [9]:
model_logistic_Reg=LogisticRegression()

model_logistic_Reg.fit(features_train,target_train) # обучение

predict_LogisticReg=model_logistic_Reg.predict(features_valid) # провка

print('accuracy:',accuracy_score(predict_LogisticReg,target_valid))

accuracy: 0.7107309486780715


Модель не переобучивается. accuracy = 0.7107309 (Самое низкое)

**Вывод:**

После обучение трёх разных моделей, были получены следующие результаты

- "Дерево предсказания" (при гиперпараметрах max_depth = 3): 0.7853810
- "Случайный лес" (при гиперпараметрах max_depth = 9, max_samples_leaf = 5 и n_estimators = 15): 0.8024883
- "Логистическая регрессия": 0.7107309

"Случайный лес" - лучшая получившаяся модель.

<a id='section4'></a>
## Проверка моделей на тестовой выборке

Тест данных на модели "**Дерево предсказаний**"

In [10]:
model_tree = DecisionTreeClassifier(random_state = 12345, max_depth = 3)

model_tree.fit(features_train, target_train)

prediction_tree = model_tree.predict(features_test)

print(' accuracy:',accuracy_score(prediction_tree,target_test))

 accuracy: 0.7853810264385692


Тест данных на модели "**Случайный лес**"

In [11]:
model_forest = RandomForestClassifier(max_depth = 9, n_estimators = 15, random_state = 12345, min_samples_leaf = 5)

model_forest.fit(features_train, target_train)

prediction_forest = model_forest.predict(features_test)

print(' accuracy:',accuracy_score(prediction_forest, target_test))

 accuracy: 0.80248833592535


Тест данных на модели "**Логистическая регрессия**"

In [12]:
model_logistic_Reg = LogisticRegression()

model_logistic_Reg.fit(features_train, target_train)

predict_LogisticReg = model_logistic_Reg.predict(features_test)

print('accuracy:',accuracy_score(predict_LogisticReg, target_test))

accuracy: 0.7107309486780715


**Вывод:**

Модели проверены на тестовой выборке. Метрики качества валидационной модели равны метрикам качества тестовой выборки.

<a id='section5'></a>
## (бонус) Проверка модели на адекватность

Проверим адекватность модели через функцию, предсказывающую значение признака is_ultra по типу "орёл-решка"

In [13]:

random_predictions = np.random.randint(low = 0, high = 2, size = 643) 

accuracy = accuracy_score(target_test, random_predictions) # доля правильных ответов

print('accuracy =','{:.7f}'.format(accuracy)) # выведем долю правильных ответов

accuracy = 0.5085537


## Общий вывод

Наиболее адекватные предсказания делает модель "Случайный лес", это видно по точности предсказания на валидационной и на тестовой выборке.